PROBABILIDAD DE DEFAULT DE LOS TITULOS ARGENTINOS

In [1]:
#Importo librerias (e instalo alguna si hace falta)
#pip install yfinance
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import seaborn as sns
import scipy.stats as scs
import statsmodels.api as sm
import scipy.optimize as sco

In [3]:
#Import the cash flows of the national bond AL30
excel = 'C:/Users/juana/OneDrive/UCEMA/Quant - Finanzas Cuantitativas/TP Final/AL30/AL30.xlsx'
sheet = 'FF'
df = pd.read_excel(excel, sheet_name=sheet, index_col='Fecha Pago')
df.index = pd.to_datetime(df.index)

#Import the prices of each day
excel = 'C:/Users/juana/OneDrive/UCEMA/Quant - Finanzas Cuantitativas/TP Final/AL30/AL30.xlsx'
sheet = 'PRICES'
prices = pd.read_excel(excel, sheet_name=sheet, index_col='date')
prices.index = pd.to_datetime(prices.index)

In [4]:
#Supuestos Iniciales
#Recovery Rate
RR = 0.4
#Estructura Temporal de Tasas de Interés
#ETTI = {'Days': range(1,10000), 'Rate': 0.4469}
ETTI = {'Days': range(1,10000), 'Rate': 0.04}
ETTI = pd.DataFrame(ETTI)
ETTI = ETTI.set_index('Days')

In [5]:
#Creo función para calcular los precios del AL30 según cada curva de tasas
def NPV(cf, date, rates):
    date = pd.to_datetime(date)
    cf2 = cf.loc[df.index > date, ['A+R']]
    cf2['Days to payment'] = (cf2.index - date).days
    cf2['Risk Free Rate'] = cf2['Days to payment'].map(rates['Rate'])
    cf2['Present Value Factor'] = (1 + cf2['Risk Free Rate'] )**-(cf2['Days to payment']/365)
    cf2['Cash Flow Present Value'] = cf2['A+R'] * cf2['Present Value Factor']
    price = sum(cf2['Cash Flow Present Value'])
    return price


In [6]:
NPV(df, '19-9-2023', ETTI )

C:\Users\juana\AppData\Local\Temp\ipykernel_18724\1157888772.py:3: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  date = pd.to_datetime(date)


89.42889874352463

In [8]:
#Armo función para calcular el valor actual de la pérdida esperada a cada fecha
def LossPV(cf, date, rates):
    date = pd.to_datetime(date)
    cf2 = cf.loc[df.index > date, ['VR','A+R']]
    cf2['Risk Free Price'] = [NPV(df, dates, rates) + cupon['A+R'] for dates, cupon in cf2.iterrows()]
    cf2['Recovery'] = cf2['VR'] * RR
    cf2['Default Loss'] = cf2['Risk Free Price'] - cf2['Recovery']
    cf2['Days to payment'] = (cf2.index - date).days
    cf2['Risk Free Rate'] = cf2['Days to payment'].map(ETTI['Rate'])
    cf2['Present Value Factor'] = (1 + cf2['Risk Free Rate'] )**-(cf2['Days to payment']/365)
    cf2['Present Value of Default Loss'] = cf2['Present Value Factor'] * cf2['Default Loss']
    PVDL = sum(cf2['Present Value of Default Loss'])
    return PVDL


LossPV(df, pd.to_datetime('19-9-2023'), ETTI )


C:\Users\juana\AppData\Local\Temp\ipykernel_18724\2775376515.py:16: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  LossPV(df, pd.to_datetime('19-9-2023'), ETTI )


408.660840582522

In [9]:
# Armo un data Frame donde voy a ir colocando cada uno de las columnas para el 
#cálculo de las probabilidades

summary = prices
summary['Risk Free Value'] = [NPV(df, dates, ETTI) for dates, prices in summary.iterrows()]
summary['Credit Spread'] = summary['Risk Free Value'] - summary['close'] 
summary['Present Value of Default Loss'] = [LossPV(df, dates, ETTI) for dates, prices in summary.iterrows()]
summary['Default Probability'] = summary['Credit Spread'] / summary['Present Value of Default Loss'] * 2  
summary


,close,Risk Free Value,Credit Spread,Present Value of Default Loss,Default Probability
date,,,,,
2020-09-14,51.870,80.623681,28.753681,577.539845,0.099573
2020-09-15,51.080,80.632345,29.552345,577.601908,0.102328
2020-09-16,47.960,80.641010,32.681010,577.663977,0.113149
2020-09-17,46.900,80.649675,33.749675,577.726052,0.116836
2020-09-18,47.100,80.658342,33.558342,577.788135,0.116161
...,...,...,...,...,...
2023-09-20,30.140,89.438509,59.298509,408.704755,0.290178
2023-09-21,29.400,89.448120,60.048120,408.748675,0.293814
2023-09-22,29.390,89.457732,60.067732,408.792599,0.293879
